# AAN_new_최종_trend+PCA+XGB.ipynb은 AAN_new 리뉴얼 데이터셋과 trend를 이용해 XGB를 학습/테스트한다.

# **앞뒤 튜플의 관계성을 trend로 나타냄.**
- sliding window 5로 A1, A2, N에 대해 5mean의 값을 추가한다.
- 이때, 5mean를 기준으로, 각 값이 위인지 아래인지를 나타낸다.
- 5개의 윈도우에서는 5개의 점으로 트렌드가 나타난다.
- 편의를 위해 3개의 점으로 근사한다.
- 이를 위해 각 윈도우 내부에서 sliding window 3을 이용한다.
- 3mean을 구한다.
- 3mean과 5mean과의 비교를 통해 up/mid/down으로 나눔
- 3개를 조합해 하나의 column trend로 만듬.

# **AAN_new 리뉴얼 데이터셋**
- 기존의 순차적 에러삽입 방식 -> 랜덤선택 에러삽입
- float64 -> float32
- PCA 0.95 -> 0.98 조정.

# **trend1 = (up/mid/down)**
- up => 5mean + 5std < 3mean[1]
- mid => 5eman - 5std < 3mean[1] < 5mean + 5std
- down => 3mean[1] < 5mean - 5std 
- 동일하게, 3mean[2], 3mean[3]에 대해서도 수행.
- 그러면 하나의 튜플에 trend1, trend2, trend3이 생김
- ex) up, up, down. 이걸 하나로(trend) 합침. ex) 113





# **데이터로드**

In [9]:
!pip install ipython-autotime
%load_ext autotime
import pandas as pd
import numpy as np
import os
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
import pickle


# read data pickle
path = os.getcwd() + "/drive/My Drive/data/AAN_new"

x_train = pd.read_pickle(path + '/reset_NoError_whole_x_train.pickle')
print('=== Data Loaded: reset_NoError_whole_x_train.pickle ===')
# y_train = pd.read_pickle(path + '/reset_NoError_whole_y_train.pickle')
# print('=== Data Loaded: reset_NoError_whole_y_train.pickle ===')
x_test = pd.read_pickle(path + '/reset_NoError_whole_x_test.pickle')
print('=== Data Loaded: reset_NoError_whole_x_test.pickle ===')
# y_test = pd.read_pickle(path + '/reset_NoError_whole_y_test.pickle')
# print('=== Data Loaded: reset_NoError_whole_y_test.pickle ===')



The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
=== Data Loaded: reset_NoError_whole_x_train.pickle ===
=== Data Loaded: reset_NoError_whole_x_test.pickle ===
time: 4.02 s


# **데이터확인**

In [0]:
x_train

,time,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,2019-11-02 09:24:00+09:00,0.0311,1.0466,406.091705,1104.010986,32.0,0.0000,30.984301,35.038799,1.47800,0.2838,100.0,96.406502,55.853600,56.787102,171.890198,829.605225,6.713800,34.956799,305.595306,1101.636597,14.19640,14.0472,407.718994,92.108902,94.361900,12.0,12.2238,1.3338,0.20840,36.184700,37.115501,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8486,-4.0689,-1.9391,-0.3011,1.2569,13496.0,-224.0,70.0,10.0,27.929701,-0.0003,-0.007200,0.108500,0.0,256.184906,-0.0013,255.298706,259.530487,0.0,10027.0,27627.0,0,0,0,0,0
1,2019-11-02 09:24:01+09:00,0.0311,1.0471,406.091705,1104.010986,32.0,0.0000,35.950100,35.039902,1.48240,0.2838,100.0,96.490501,55.768002,56.794701,173.034698,830.909729,5.378700,28.005100,306.930511,1101.955444,14.21220,14.0472,407.718994,91.969002,94.361900,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.021200,1.0,0.0,1.0,0.0,792.0,0.0,11.7642,-4.7702,-2.1559,-0.1027,1.6715,13512.0,-224.0,70.0,10.0,28.436100,0.0000,-0.007200,0.108500,0.0,256.564697,0.0001,257.559296,258.409210,0.0,10030.0,27627.0,0,0,0,0,0
2,2019-11-02 09:24:02+09:00,0.0311,1.0410,406.091705,1104.010986,32.0,0.0000,34.956799,35.024601,1.58160,0.2838,100.0,96.490501,55.874500,56.787102,170.936600,833.518982,7.667500,39.922100,305.404602,1104.187866,14.12440,14.0472,407.821198,91.835899,94.453400,12.0,12.2238,1.3330,0.20690,36.184700,37.115501,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6485,-4.6927,-1.8849,0.6518,1.9967,13528.0,-240.0,70.0,10.0,28.920700,0.0000,-0.007200,0.054200,0.0,257.360413,-0.0001,256.022186,258.770996,0.0,10026.0,27627.0,0,0,0,0,0
3,2019-11-02 09:24:03+09:00,0.0311,1.0430,406.091705,1104.010986,32.0,0.0000,28.005100,35.029598,1.38400,0.2838,100.0,96.490501,55.736599,56.794701,174.751297,828.735413,6.523100,33.963799,307.502686,1101.636597,14.20600,14.0472,407.667786,91.657204,94.369499,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.6964,-3.9879,-1.5550,0.8548,1.7724,13544.0,-240.0,70.0,10.0,29.253500,0.0000,-0.007200,0.054200,0.0,257.107208,0.0007,256.148712,258.445404,0.0,10027.0,27627.0,0,0,0,0,0
4,2019-11-02 09:24:04+09:00,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432922,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,-1.6621,0.1528,1.3034,13560.0,-240.0,70.0,10.0,29.528400,-0.0003,-0.007200,0.054200,0.0,254.882797,-0.0013,255.606201,258.517792,0.0,10030.0,27627.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727193,2019-11-01 19:59:55+09:00,0.0998,1.3857,404.178711,1040.812500,32.0,9.5426,156.114197,35.714699,11.71167,11.1862,100.0,96.375999,51.506550,52.773998,157.108276,796.989502,29.363649,153.631363,270.118652,1042.957397,12.22482,11.3235,403.629608,77.613701,78.996300,12.0,12.2238,1.3887,0.17505,35.482800,36.703499,28.022900,1.0,0.0,1.0,0.0,792.0,0.0,11.7158,-3.8670,-1.5737,0.6780,1.5849,2888.0,12064.0,70.0,10.0,18.478701,0.0010,64.525497,54.922600,0.0,393.681305,-0.0031,329.029205,329.553589,0.0,10053.0,27617.0,0,0,0,0,0
727194,2019-11-01 19:59:56+09:00,0.

time: 164 ms


In [0]:
x_test

,time,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error
0,2019-11-01 20:00:00+09:00,0.0998,1.3901,404.178711,1040.812500,32.0,9.672400,149.162598,35.725800,12.025100,11.186200,100.0,96.375999,51.389400,52.773998,157.966599,798.294128,28.648399,154.128006,270.309387,1043.276367,12.151700,11.3235,403.834015,77.626404,79.003899,12.0,12.2238,1.3872,0.1733,35.482800,36.703499,28.028999,1.0,0.0,1.0,0.0,791.0,0.0,11.7120,-3.8775,-1.5711,0.7068,1.6114,2232.0,12784.0,70.0,10.0,18.301500,0.0007,65.386299,58.828899,0.0,392.993988,0.0005,327.600494,329.336700,0.0,10053.0,27618.0,0,0,0,0,0
1,2019-11-01 20:00:01+09:00,0.0998,1.3840,404.178711,1040.812500,32.0,9.696700,150.155502,35.710499,12.156200,11.186200,100.0,96.375999,51.393002,52.773998,155.487000,799.163818,29.029900,149.162598,269.737213,1042.957397,12.136500,11.3235,403.680695,77.599503,79.003899,12.0,12.2238,1.3864,0.1779,35.498001,36.688202,28.019400,1.0,0.0,1.0,0.0,793.0,0.0,11.8256,-3.7916,-1.7599,-0.0161,1.2254,2120.0,12896.0,70.0,10.0,18.236401,0.0005,65.407997,59.570301,0.0,394.223785,-0.0027,325.882507,329.842987,0.0,10053.0,27617.0,0,0,0,0,0
2,2019-11-01 20:00:02+09:00,0.0998,1.3837,404.178711,1040.812500,32.0,9.704100,152.141693,35.709801,12.180500,11.186200,100.0,96.375999,51.424599,52.773998,159.110992,793.510376,29.029900,151.148804,269.927887,1042.000732,12.198100,11.3235,403.782898,77.600304,78.996300,12.0,12.2238,1.3872,0.1749,35.482800,36.688202,28.024700,1.0,0.0,1.0,0.0,792.0,0.0,11.8218,-4.4148,-2.1085,-0.3602,1.3557,2016.0,12976.0,70.0,10.0,18.268999,0.0007,65.357399,60.004299,0.0,391.167511,0.0008,325.900604,328.540894,0.0,10053.0,27616.0,0,0,0,0,0
3,2019-11-01 20:00:03+09:00,0.0998,1.3854,404.178711,1040.812500,32.0,9.715200,154.128006,35.714001,12.247300,11.186200,100.0,96.375999,51.406399,52.773998,156.059296,801.773193,30.174200,151.148804,271.263092,1042.319580,12.170300,11.3235,403.782898,77.603798,79.003899,12.0,12.2238,1.3879,0.1772,35.482800,36.688202,28.024599,1.0,0.0,1.0,0.0,792.0,0.0,11.7147,-4.9002,-2.1456,0.1541,1.8032,1912.0,13056.0,70.0,10.0,18.243601,0.0005,65.328400,60.474499,0.0,390.606903,-0.0001,325.159088,328.215393,0.0,10053.0,27618.0,0,0,0,0,0
4,2019-11-01 20:00:04+09:00,0.0998,1.3839,404.178711,1040.812500,32.0,9.748800,149.162598,35.710098,12.209800,11.186200,100.0,96.375999,51.306599,52.773998,155.868500,794.815186,29.411301,157.107101,271.072388,1044.551880,12.079600,11.3235,403.578491,77.590897,78.996300,12.0,12.2162,1.3872,0.1787,35.482800,36.688202,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6509,-4.5318,-1.7957,0.8014,1.9603,1816.0,13136.0,70.0,10.0,18.225500,0.0007,65.263298,60.854301,0.0,390.136688,0.0007,324.688995,326.931396,0.0,10053.0,27616.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181795,2019-10-30 03:11:55+09:00,0.0305,1.3269,392.330292,1115.077148,32.0,84.305901,2556.180908,35.741001,40.301601,40.284698,0.0,-1.852400,58.626598,59.030201,185.050903,860.916077,1616.897583,2563.885010,312.271088,1113.117310,17.037399,16.8930,391.054810,42.979401,43.557701,12.0,11.6287,1.2697,0.7166,35.223400,35.543800,28.020300,1.0,0.0,1.0,0.0,791.0,0.0,11.6964,-5.0314,-2.2791,0.0256,1.7013,7464.0,1712.0,70.0,10.0,27.112301,-0.0009,1.511900,9.403900,0.0,335.159912,0.0011,329.788788,331.072815,0.0,1002

time: 113 ms


# **테스트**

In [0]:
from pandas import Series, DataFrame

testvar = {'value':  [1, 2, 3, 4, 5, 6, 5, 4, 3, 2]}
testdf = DataFrame(testvar)
testdf['5mean'] = testdf['value'].rolling(window=5).mean()
testdf['5std'] = testdf['value'].rolling(window=5).std()
testdf['3mean'] = testdf['value'].rolling(window=3).mean()

# rolling NaN 제거
testdf.drop(testdf.head(2).index, inplace=True)
testdf.reset_index(drop=True, inplace=True)

testdf



,value,5mean,5std,3mean
0,3,NaN,NaN,2.000000
1,4,NaN,NaN,3.000000
2,5,3.0,1.581139,4.000000
3,6,4.0,1.581139,5.000000
4,5,4.6,1.140175,5.333333
5,4,4.8,0.836660,5.000000
6,3,4.6,1.140175,4.000000
7,2,4.0,1.581139,3.000000


time: 34.7 ms


In [0]:
# 한줄로 만듬.
lastindex = 7
testdf['3mean1'] = 999.0
testdf['3mean2'] = 999.0
testdf['3mean3'] = testdf['3mean']  # 보기편하게


df = testdf
df2 = testdf

ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
testdf['3mean1'] = df['3mean3']
testdf['3mean1'] = df['3mean3']
testdf['3mean1'] = df['3mean3']


df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
testdf['3mean2'] = df2['3mean3']
testdf['3mean2'] = df2['3mean3']
testdf['3mean2'] = df2['3mean3']

testdf





,value,5mean,5std,3mean,3mean1,3mean2,3mean3
0,3,NaN,NaN,2.000000,NaN,NaN,2.000000
1,4,NaN,NaN,3.000000,NaN,NaN,3.000000
2,5,3.0,1.581139,4.000000,2.000000,3.000000,4.000000
3,6,4.0,1.581139,5.000000,3.000000,4.000000,5.000000
4,5,4.6,1.140175,5.333333,4.000000,5.000000,5.333333
5,4,4.8,0.836660,5.000000,5.000000,5.333333,5.000000
6,3,4.6,1.140175,4.000000,5.333333,5.000000,4.000000
7,2,4.0,1.581139,3.000000,5.000000,4.000000,3.000000


time: 49.6 ms


In [0]:
testdf['bound_up'] = testdf['5mean'] + (testdf['5std'] /2)
testdf['bound_down'] = testdf['5mean'] - (testdf['5std'] /2)

# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
testdf = testdf.apply(lambda x: pd.Series.round(x, 5))
testdf

,value,5mean,5std,3mean,3mean1,3mean2,3mean3,bound_up,bound_down
0,3,NaN,NaN,2.00000,NaN,NaN,2.00000,NaN,NaN
1,4,NaN,NaN,3.00000,NaN,NaN,3.00000,NaN,NaN
2,5,3.0,1.58114,4.00000,2.00000,3.00000,4.00000,3.79057,2.20943
3,6,4.0,1.58114,5.00000,3.00000,4.00000,5.00000,4.79057,3.20943
4,5,4.6,1.14018,5.33333,4.00000,5.00000,5.33333,5.17009,4.02991
5,4,4.8,0.83666,5.00000,5.00000,5.33333,5.00000,5.21833,4.38167
6,3,4.6,1.14018,4.00000,5.33333,5.00000,4.00000,5.17009,4.02991
7,2,4.0,1.58114,3.00000,5.00000,4.00000,3.00000,4.79057,3.20943


time: 37.8 ms


In [0]:
testdf['trend1'] = 999
testdf['trend2'] = 999
testdf['trend3'] = 999

# bound NaN 제거
testdf.drop(testdf.head(2).index, inplace=True)
print("")

def cal_trend(df, boundDown, boundUp, threeMean1, threeMean2, threeMean3):
  df.loc[(threeMean1 >= boundDown) & (threeMean1 <= boundUp), 'trend1'] = 2
  df.loc[(threeMean1 < boundDown), 'trend1'] = 1
  df.loc[(threeMean1 > boundUp), 'trend1'] = 3

  df.loc[(threeMean2 >= boundDown) & (threeMean2 <= boundUp), 'trend2'] = 2
  df.loc[(threeMean2 < boundDown), 'trend2'] = 1
  df.loc[(threeMean2 > boundUp), 'trend2'] = 3

  df.loc[(threeMean3 >= boundDown) & (threeMean3 <= boundUp), 'trend3'] = 2
  df.loc[(threeMean3 < boundDown), 'trend3'] = 1
  df.loc[(threeMean3 > boundUp), 'trend3'] = 3

cal_trend(testdf, testdf['bound_down'], testdf['bound_up'], testdf['3mean1'], testdf['3mean2'], testdf['3mean3'])

df = testdf
testdf




,value,5mean,5std,3mean,3mean1,3mean2,3mean3,bound_up,bound_down,trend1,trend2,trend3
2,5,3.0,1.58114,4.00000,2.00000,3.00000,4.00000,3.79057,2.20943,1,2,3
3,6,4.0,1.58114,5.00000,3.00000,4.00000,5.00000,4.79057,3.20943,1,2,3
4,5,4.6,1.14018,5.33333,4.00000,5.00000,5.33333,5.17009,4.02991,1,2,3
5,4,4.8,0.83666,5.00000,5.00000,5.33333,5.00000,5.21833,4.38167,2,3,2
6,3,4.6,1.14018,4.00000,5.33333,5.00000,4.00000,5.17009,4.02991,3,2,1
7,2,4.0,1.58114,3.00000,5.00000,4.00000,3.00000,4.79057,3.20943,3,2,1


time: 56.2 ms


In [0]:
# trend123을 trend로 합치기
# 111, 112, 113, ... 332, 333 순으로 1-27번호.
df.loc[((df.trend1 == 1) & (df.trend2 == 1) & (df.trend3 == 1)), 'trend'] = 1
df.loc[((df.trend1 == 1) & (df.trend2 == 1) & (df.trend3 == 2)), 'trend'] = 2
df.loc[((df.trend1 == 1) & (df.trend2 == 1) & (df.trend3 == 3)), 'trend'] = 3
df.loc[((df.trend1 == 1) & (df.trend2 == 2) & (df.trend3 == 1)), 'trend'] = 4
df.loc[((df.trend1 == 1) & (df.trend2 == 2) & (df.trend3 == 2)), 'trend'] = 5
df.loc[((df.trend1 == 1) & (df.trend2 == 2) & (df.trend3 == 3)), 'trend'] = 6
df.loc[((df.trend1 == 1) & (df.trend2 == 3) & (df.trend3 == 1)), 'trend'] = 7
df.loc[((df.trend1 == 1) & (df.trend2 == 3) & (df.trend3 == 2)), 'trend'] = 8
df.loc[((df.trend1 == 1) & (df.trend2 == 3) & (df.trend3 == 3)), 'trend'] = 9

df.loc[((df.trend1 == 2) & (df.trend2 == 1) & (df.trend3 == 1)), 'trend'] = 10
df.loc[((df.trend1 == 2) & (df.trend2 == 1) & (df.trend3 == 2)), 'trend'] = 11
df.loc[((df.trend1 == 2) & (df.trend2 == 1) & (df.trend3 == 3)), 'trend'] = 12
df.loc[((df.trend1 == 2) & (df.trend2 == 2) & (df.trend3 == 1)), 'trend'] = 13
df.loc[((df.trend1 == 2) & (df.trend2 == 2) & (df.trend3 == 2)), 'trend'] = 14
df.loc[((df.trend1 == 2) & (df.trend2 == 2) & (df.trend3 == 3)), 'trend'] = 15
df.loc[((df.trend1 == 2) & (df.trend2 == 3) & (df.trend3 == 1)), 'trend'] = 16
df.loc[((df.trend1 == 2) & (df.trend2 == 3) & (df.trend3 == 2)), 'trend'] = 17
df.loc[((df.trend1 == 2) & (df.trend2 == 3) & (df.trend3 == 3)), 'trend'] = 18

df.loc[((df.trend1 == 3) & (df.trend2 == 1) & (df.trend3 == 1)), 'trend'] = 19
df.loc[((df.trend1 == 3) & (df.trend2 == 1) & (df.trend3 == 2)), 'trend'] = 20
df.loc[((df.trend1 == 3) & (df.trend2 == 1) & (df.trend3 == 3)), 'trend'] = 21
df.loc[((df.trend1 == 3) & (df.trend2 == 2) & (df.trend3 == 1)), 'trend'] = 22
df.loc[((df.trend1 == 3) & (df.trend2 == 2) & (df.trend3 == 2)), 'trend'] = 23
df.loc[((df.trend1 == 3) & (df.trend2 == 2) & (df.trend3 == 3)), 'trend'] = 24
df.loc[((df.trend1 == 3) & (df.trend2 == 3) & (df.trend3 == 1)), 'trend'] = 25
df.loc[((df.trend1 == 3) & (df.trend2 == 3) & (df.trend3 == 2)), 'trend'] = 26
df.loc[((df.trend1 == 3) & (df.trend2 == 3) & (df.trend3 == 3)), 'trend'] = 27

df

,value,5mean,5std,3mean,3mean1,3mean2,3mean3,bound_up,bound_down,trend1,trend2,trend3,trend
2,5,3.0,1.58114,4.00000,2.00000,3.00000,4.00000,3.79057,2.20943,1,2,3,6.0
3,6,4.0,1.58114,5.00000,3.00000,4.00000,5.00000,4.79057,3.20943,1,2,3,6.0
4,5,4.6,1.14018,5.33333,4.00000,5.00000,5.33333,5.17009,4.02991,1,2,3,6.0
5,4,4.8,0.83666,5.00000,5.00000,5.33333,5.00000,5.21833,4.38167,2,3,2,17.0
6,3,4.6,1.14018,4.00000,5.33333,5.00000,4.00000,5.17009,4.02991,3,2,1,22.0
7,2,4.0,1.58114,3.00000,5.00000,4.00000,3.00000,4.79057,3.20943,3,2,1,22.0


time: 124 ms


# **rolling 5mean, 5std, 3mean 추가**
- A1, A2, N에 대해

In [10]:
x_train.drop(['time'], axis=1, inplace=True)

# x_train의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_train['A1_5min'] = x_train['P1.B3004'].rolling(window=5).min()
x_train['A1_5max'] = x_train['P1.B3004'].rolling(window=5).max()
x_train['A1_5mean'] = x_train['P1.B3004'].rolling(window=5).mean()
x_train['A1_5std'] = x_train['P1.B3004'].rolling(window=5).std()
x_train['A1_3mean'] = x_train['P1.B3004'].rolling(window=3).mean()

x_train['A2_5min'] = x_train['P1.B3005'].rolling(window=5).min()
x_train['A2_5max'] = x_train['P1.B3005'].rolling(window=5).max()
x_train['A2_5mean'] = x_train['P1.B3005'].rolling(window=5).mean()
x_train['A2_5std'] = x_train['P1.B3005'].rolling(window=5).std()
x_train['A2_3mean'] = x_train['P1.B3005'].rolling(window=3).mean()

x_train['N_5min'] = x_train['P1.B4002'].rolling(window=5).min()
x_train['N_5max'] = x_train['P1.B4002'].rolling(window=5).max()
x_train['N_5mean'] = x_train['P1.B4002'].rolling(window=5).mean()
x_train['N_5std'] = x_train['P1.B4002'].rolling(window=5).std()
x_train['N_3mean'] = x_train['P1.B4002'].rolling(window=3).mean()

# rolling NaN 제거
x_train.drop(x_train.head(2).index, inplace=True)
x_train.reset_index(drop=True, inplace=True)

x_train



,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,P2.VXT03,P2.VYT02,P2.VYT03,P3.LCP01D,P3.LCV01D,P3.LH,P3.LL,P3.LT01,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error,A1_5min,A1_5max,A1_5mean,A1_5std,A1_3mean,A2_5min,A2_5max,A2_5mean,A2_5std,A2_3mean,N_5min,N_5max,N_5mean,N_5std,N_3mean
0,0.0311,1.0410,406.091705,1104.010986,32.0,0.0000,34.956799,35.024601,1.58160,0.2838,100.0,96.490501,55.874500,56.787102,170.936600,833.518982,7.667500,39.922100,305.404602,1104.187866,14.12440,14.0472,407.821198,91.835899,94.453400,12.0,12.2238,1.3330,0.20690,36.184700,37.115501,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6485,-4.6927,-1.8849,0.6518,1.9967,13528.0,-240.0,70.0,10.0,28.920700,0.0000,-0.007200,0.054200,0.0,257.360413,-0.0001,256.022186,258.770996,0.0,10026.0,27627.0,0,0,0,0,0,NaN,NaN,NaN,NaN,406.091705,NaN,NaN,NaN,NaN,1104.010986,NaN,NaN,NaN,NaN,32.0
1,0.0311,1.0430,406.091705,1104.010986,32.0,0.0000,28.005100,35.029598,1.38400,0.2838,100.0,96.490501,55.736599,56.794701,174.751297,828.735413,6.523100,33.963799,307.502686,1101.636597,14.20600,14.0472,407.667786,91.657204,94.369499,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.6964,-3.9879,-1.5550,0.8548,1.7724,13544.0,-240.0,70.0,10.0,29.253500,0.0000,-0.007200,0.054200,0.0,257.107208,0.0007,256.148712,258.445404,0.0,10027.0,27627.0,0,0,0,0,0,NaN,NaN,NaN,NaN,406.091705,NaN,NaN,NaN,NaN,1104.010986,NaN,NaN,NaN,NaN,32.0
2,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432922,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,-1.6621,0.1528,1.3034,13560.0,-240.0,70.0,10.0,29.528400,-0.0003,-0.007200,0.054200,0.0,254.882797,-0.0013,255.606201,258.517792,0.0,10030.0,27627.0,0,0,0,0,0,406.091705,406.091705,406.091705,0.0,406.091705,1104.010986,1104.010986,1104.010986,0.0,1104.010986,32.0,32.0,32.0,0.000000e+00,32.0
3,0.0311,1.0434,406.091705,1104.010986,32.0,0.0000,33.963799,35.030701,1.45520,0.2838,100.0,96.414200,55.822601,56.794701,172.653198,830.039978,5.950900,30.984301,306.167603,1104.825562,14.11300,14.0472,407.463287,91.369003,93.415802,12.0,12.2238,1.3330,0.20920,36.184700,37.115501,28.027300,1.0,0.0,1.0,0.0,793.0,0.0,11.8339,-4.2946,-2.0438,-0.3449,1.3325,13568.0,-240.0,70.0,10.0,29.738100,0.0000,-0.007200,0.054200,0.0,255.750900,0.0018,255.280701,257.902893,0.0,10027.0,27627.0,0,0,0,0,0,406.091705,406.091705,406.091705,0.0,406.091705,1104.010986,1104.010986,1104.010986,0.0,1104.010986,32.0,32.0,32.0,0.000000e+00,32.0
4,0.0311,1.0378,406.091705,1104.010986,32.0,0.0000,33.963799,35.016602,1.49180,0.2838,100.0,96.490501,55.693001,56.794701,174.369797,832.649292,7.095300,36.943001,308.074890,1102.912231,14.20050,14.0472,407.514496,91.233597,93.301399,12.0,12.2238,1.3330,0.20230,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.7272,-4.8579,-2.1167,0.1332,1.8205,13576.0,-240.0,70.0,10.0,29.966000,0.0000,-0.007200,0.054200,0.0,253.978607,0.0033,253.779694,257.378510,0.0,10027.0,27627.0,0,0,0,0,0,406.091705,406.091705,406.091705,0.0,406.091705,1104.010986,1104.010986,1104.010986,0.0,1104.010986,32.0,32.0,32.0,0.000000e+00,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

time: 1.02 s


# **3mean1, 3mean2, 3mean3을 추가**
- window 5에서 첫 3개, 두번째 3개, 마지막 3개의 mean


In [11]:
# 한줄로 만듬.
lastindex = 727195
x_train['A1_3mean1'] = 999.0
x_train['A1_3mean2'] = 999.0
x_train['A1_3mean3'] = x_train['A1_3mean']  # 보기편하게
x_train['A2_3mean1'] = 999.0
x_train['A2_3mean2'] = 999.0
x_train['A2_3mean3'] = x_train['A2_3mean']  # 보기편하게
x_train['N_3mean1'] = 999.0
x_train['N_3mean2'] = 999.0
x_train['N_3mean3'] = x_train['N_3mean']  # 보기편하게


# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_train
df2 = x_train
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_train['A1_3mean1'] = df['A1_3mean3']
x_train['A2_3mean1'] = df['A2_3mean3']
x_train['N_3mean1'] = df['N_3mean3']

df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_train['A1_3mean2'] = df2['A1_3mean3']
x_train['A2_3mean2'] = df2['A2_3mean3']
x_train['N_3mean2'] = df2['N_3mean3']


x_train


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,...,P4.HT_FD,P4.HT_LD,P4.HT_PO,P4.HT_PS,P4.LD,P4.ST_FD,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error,A1_5min,A1_5max,A1_5mean,A1_5std,A1_3mean,A2_5min,A2_5max,A2_5mean,A2_5std,A2_3mean,N_5min,N_5max,N_5mean,N_5std,N_3mean,A1_3mean1,A1_3mean2,A1_3mean3,A2_3mean1,A2_3mean2,A2_3mean3,N_3mean1,N_3mean2,N_3mean3
0,0.0311,1.0410,406.091705,1104.010986,32.0,0.0000,34.956799,35.024601,1.58160,0.2838,100.0,96.490501,55.874500,56.787102,170.936600,833.518982,7.667500,39.922100,305.404602,1104.187866,14.12440,14.0472,407.821198,91.835899,94.453400,12.0,12.2238,1.3330,0.20690,36.184700,37.115501,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6485,-4.6927,...,0.0000,-0.007200,0.054200,0.0,257.360413,-0.0001,256.022186,258.770996,0.0,10026.0,27627.0,0,0,0,0,0,NaN,NaN,NaN,NaN,406.091705,NaN,NaN,NaN,NaN,1104.010986,NaN,NaN,NaN,NaN,32.0,NaN,NaN,406.091705,NaN,NaN,1104.010986,NaN,NaN,32.0
1,0.0311,1.0430,406.091705,1104.010986,32.0,0.0000,28.005100,35.029598,1.38400,0.2838,100.0,96.490501,55.736599,56.794701,174.751297,828.735413,6.523100,33.963799,307.502686,1101.636597,14.20600,14.0472,407.667786,91.657204,94.369499,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.6964,-3.9879,...,0.0000,-0.007200,0.054200,0.0,257.107208,0.0007,256.148712,258.445404,0.0,10027.0,27627.0,0,0,0,0,0,NaN,NaN,NaN,NaN,406.091705,NaN,NaN,NaN,NaN,1104.010986,NaN,NaN,NaN,NaN,32.0,NaN,NaN,406.091705,NaN,NaN,1104.010986,NaN,NaN,32.0
2,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432922,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,...,-0.0003,-0.007200,0.054200,0.0,254.882797,-0.0013,255.606201,258.517792,0.0,10030.0,27627.0,0,0,0,0,0,406.091705,406.091705,406.091705,0.0,406.091705,1104.010986,1104.010986,1104.010986,0.0,1104.010986,32.0,32.0,32.0,0.000000e+00,32.0,406.091705,406.091705,406.091705,1104.010986,1104.010986,1104.010986,32.0,32.0,32.0
3,0.0311,1.0434,406.091705,1104.010986,32.0,0.0000,33.963799,35.030701,1.45520,0.2838,100.0,96.414200,55.822601,56.794701,172.653198,830.039978,5.950900,30.984301,306.167603,1104.825562,14.11300,14.0472,407.463287,91.369003,93.415802,12.0,12.2238,1.3330,0.20920,36.184700,37.115501,28.027300,1.0,0.0,1.0,0.0,793.0,0.0,11.8339,-4.2946,...,0.0000,-0.007200,0.054200,0.0,255.750900,0.0018,255.280701,257.902893,0.0,10027.0,27627.0,0,0,0,0,0,406.091705,406.091705,406.091705,0.0,406.091705,1104.010986,1104.010986,1104.010986,0.0,1104.010986,32.0,32.0,32.0,0.000000e+00,32.0,406.091705,406.091705,406.091705,1104.010986,1104.010986,1104.010986,32.0,32.0,32.0
4,0.0311,1.0378,406.091705,1104.010986,32.0,0.0000,33.963799,35.016602,1.49180,0.2838,100.0,96.490501,55.693001,56.794701,174.369797,832.649292,7.095300,36.943001,308.074890,1102.912231,14.20050,14.0472,407.514496,91.233597,93.301399,12.0,12.2238,1.3330,0.20230,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.7272,-4.8579,...,0.0000,-0.007200,0.054200,0.0,253.978607,0.0033,253.779694,257.378510,0.0,10027.0,27627.0,0,0,0,0,0,406.091705,406.091705,406.091705,0.0,406.091705,1104.010986,1104.010986,1104.010986,0.0,1104.010986,32.0,32.0,32.0,0.000000e+00,32.0,406.091705,406.091705,406.091705,1104.010986,1104.010986,1104.010986,32.0,32.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

time: 989 ms


# **trend 계산을 위한 up, down bound 계산**

In [12]:
# bound_up, bound_down 계산
x_train['A1_bound_up'] = x_train['A1_5mean'] + x_train['A1_5std']/2 
x_train['A1_bound_down'] = x_train['A1_5mean'] - x_train['A1_5std']/2 
x_train['A2_bound_up'] = x_train['A2_5mean'] + x_train['A2_5std']/2 
x_train['A2_bound_down'] = x_train['A2_5mean'] - x_train['A2_5std']/2 
x_train['N_bound_up'] = x_train['N_5mean'] + x_train['N_5std']/2 
x_train['N_bound_down'] = x_train['N_5mean'] - x_train['N_5std']/2

# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_train = x_train.apply(lambda x: pd.Series.round(x, 5))
x_train


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,...,P4.ST_LD,P4.ST_PO,P4.ST_PS,P4.ST_PT01,P4.ST_TT01,attack,attack.P1,attack.P2,attack.P3,error,A1_5min,A1_5max,A1_5mean,A1_5std,A1_3mean,A2_5min,A2_5max,A2_5mean,A2_5std,A2_3mean,N_5min,N_5max,N_5mean,N_5std,N_3mean,A1_3mean1,A1_3mean2,A1_3mean3,A2_3mean1,A2_3mean2,A2_3mean3,N_3mean1,N_3mean2,N_3mean3,A1_bound_up,A1_bound_down,A2_bound_up,A2_bound_down,N_bound_up,N_bound_down
0,0.0311,1.0410,406.091705,1104.010986,32.0,0.0000,34.956799,35.024601,1.58160,0.2838,100.0,96.490501,55.874500,56.787102,170.936600,833.518982,7.667500,39.922100,305.404602,1104.187866,14.12440,14.0472,407.821198,91.835899,94.453400,12.0,12.2238,1.3330,0.20690,36.184700,37.115501,28.022100,1.0,0.0,1.0,0.0,792.0,0.0,11.6485,-4.6927,...,256.022186,258.770996,0.0,10026.0,27627.0,0,0,0,0,0,NaN,NaN,NaN,NaN,406.09171,NaN,NaN,NaN,NaN,1104.01099,NaN,NaN,NaN,NaN,32.0,NaN,NaN,406.09171,NaN,NaN,1104.01099,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0311,1.0430,406.091705,1104.010986,32.0,0.0000,28.005100,35.029598,1.38400,0.2838,100.0,96.490501,55.736599,56.794701,174.751297,828.735413,6.523100,33.963799,307.502686,1101.636597,14.20600,14.0472,407.667786,91.657204,94.369499,12.0,12.2238,1.3322,0.20460,36.184700,37.115501,28.020300,1.0,0.0,1.0,0.0,792.0,0.0,11.6964,-3.9879,...,256.148712,258.445404,0.0,10027.0,27627.0,0,0,0,0,0,NaN,NaN,NaN,NaN,406.09171,NaN,NaN,NaN,NaN,1104.01099,NaN,NaN,NaN,NaN,32.0,NaN,NaN,406.09171,NaN,NaN,1104.01099,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432983,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,...,255.606201,258.517792,0.0,10030.0,27627.0,0,0,0,0,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0
3,0.0311,1.0434,406.091705,1104.010986,32.0,0.0000,33.963799,35.030701,1.45520,0.2838,100.0,96.414200,55.822601,56.794701,172.653198,830.039978,5.950900,30.984301,306.167603,1104.825562,14.11300,14.0472,407.463287,91.369003,93.415802,12.0,12.2238,1.3330,0.20920,36.184700,37.115501,28.027300,1.0,0.0,1.0,0.0,793.0,0.0,11.8339,-4.2946,...,255.280701,257.902893,0.0,10027.0,27627.0,0,0,0,0,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0
4,0.0311,1.0378,406.091705,1104.010986,32.0,0.0000,33.963799,35.016602,1.49180,0.2838,100.0,96.490501,55.693001,56.794701,174.369797,832.649292,7.095300,36.943001,308.074890,1102.912231,14.20050,14.0472,407.514465,91.233597,93.301399,12.0,12.2238,1.3330,0.20230,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.7272,-4.8579,...,253.779694,257.378510,0.0,10027.0,27627.0,0,0,0,0,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

time: 1.24 s


# **trend1,2,3 계산**

In [13]:
x_train['A1_trend1'] = 999
x_train['A1_trend2'] = 999
x_train['A1_trend3'] = 999
x_train['A2_trend1'] = 999
x_train['A2_trend2'] = 999
x_train['A2_trend3'] = 999
x_train['N_trend1'] = 999
x_train['N_trend2'] = 999
x_train['N_trend3'] = 999

# NaN으로 못쓰는 데이터 제거
x_train.drop(x_train.head(2).index, inplace=True)
print("")

# A1의 trend1,2,3 구하는 함수
def cal_trendA1(df, boundDown, boundUp, threeMean1, threeMean2, threeMean3):
  df.loc[(threeMean1 >= boundDown) & (threeMean1 <= boundUp), 'A1_trend1'] = 2
  df.loc[(threeMean1 < boundDown), 'A1_trend1'] = 1
  df.loc[(threeMean1 > boundUp), 'A1_trend1'] = 3
  df.loc[(threeMean2 >= boundDown) & (threeMean2 <= boundUp), 'A1_trend2'] = 2
  df.loc[(threeMean2 < boundDown), 'A1_trend2'] = 1
  df.loc[(threeMean2 > boundUp), 'A1_trend2'] = 3
  df.loc[(threeMean3 >= boundDown) & (threeMean3 <= boundUp), 'A1_trend3'] = 2
  df.loc[(threeMean3 < boundDown), 'A1_trend3'] = 1
  df.loc[(threeMean3 > boundUp), 'A1_trend3'] = 3
#A2의 trend1,2,3 구하는 함수
def cal_trendA2(df, boundDown, boundUp, threeMean1, threeMean2, threeMean3):
  df.loc[(threeMean1 >= boundDown) & (threeMean1 <= boundUp), 'A2_trend1'] = 2
  df.loc[(threeMean1 < boundDown), 'A2_trend1'] = 1
  df.loc[(threeMean1 > boundUp), 'A2_trend1'] = 3
  df.loc[(threeMean2 >= boundDown) & (threeMean2 <= boundUp), 'A2_trend2'] = 2
  df.loc[(threeMean2 < boundDown), 'A2_trend2'] = 1
  df.loc[(threeMean2 > boundUp), 'A2_trend2'] = 3
  df.loc[(threeMean3 >= boundDown) & (threeMean3 <= boundUp), 'A2_trend3'] = 2
  df.loc[(threeMean3 < boundDown), 'A2_trend3'] = 1
  df.loc[(threeMean3 > boundUp), 'A2_trend3'] = 3
# N의 trend1,2,3 구하는 함수
def cal_trendN(df, boundDown, boundUp, threeMean1, threeMean2, threeMean3):
  df.loc[(threeMean1 >= boundDown) & (threeMean1 <= boundUp), 'N_trend1'] = 2
  df.loc[(threeMean1 < boundDown), 'N_trend1'] = 1
  df.loc[(threeMean1 > boundUp), 'N_trend1'] = 3
  df.loc[(threeMean2 >= boundDown) & (threeMean2 <= boundUp), 'N_trend2'] = 2
  df.loc[(threeMean2 < boundDown), 'N_trend2'] = 1
  df.loc[(threeMean2 > boundUp), 'N_trend2'] = 3
  df.loc[(threeMean3 >= boundDown) & (threeMean3 <= boundUp), 'N_trend3'] = 2
  df.loc[(threeMean3 < boundDown), 'N_trend3'] = 1
  df.loc[(threeMean3 > boundUp), 'N_trend3'] = 3

# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_train, x_train['A1_bound_down'], x_train['A1_bound_up'], x_train['A1_3mean1'], x_train['A1_3mean2'], x_train['A1_3mean3'])
cal_trendA2(x_train, x_train['A2_bound_down'], x_train['A2_bound_up'], x_train['A2_3mean1'], x_train['A2_3mean2'], x_train['A2_3mean3'])
cal_trendN(x_train, x_train['N_bound_down'], x_train['N_bound_up'], x_train['N_3mean1'], x_train['N_3mean2'], x_train['N_3mean3'])

x_train


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,...,error,A1_5min,A1_5max,A1_5mean,A1_5std,A1_3mean,A2_5min,A2_5max,A2_5mean,A2_5std,A2_3mean,N_5min,N_5max,N_5mean,N_5std,N_3mean,A1_3mean1,A1_3mean2,A1_3mean3,A2_3mean1,A2_3mean2,A2_3mean3,N_3mean1,N_3mean2,N_3mean3,A1_bound_up,A1_bound_down,A2_bound_up,A2_bound_down,N_bound_up,N_bound_down,A1_trend1,A1_trend2,A1_trend3,A2_trend1,A2_trend2,A2_trend3,N_trend1,N_trend2,N_trend3
2,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432983,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,...,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2
3,0.0311,1.0434,406.091705,1104.010986,32.0,0.0000,33.963799,35.030701,1.45520,0.2838,100.0,96.414200,55.822601,56.794701,172.653198,830.039978,5.950900,30.984301,306.167603,1104.825562,14.11300,14.0472,407.463287,91.369003,93.415802,12.0,12.2238,1.3330,0.20920,36.184700,37.115501,28.027300,1.0,0.0,1.0,0.0,793.0,0.0,11.8339,-4.2946,...,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2
4,0.0311,1.0378,406.091705,1104.010986,32.0,0.0000,33.963799,35.016602,1.49180,0.2838,100.0,96.490501,55.693001,56.794701,174.369797,832.649292,7.095300,36.943001,308.074890,1102.912231,14.20050,14.0472,407.514465,91.233597,93.301399,12.0,12.2238,1.3330,0.20230,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.7272,-4.8579,...,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2
5,0.0311,1.0341,406.091705,1104.010986,32.0,0.0000,30.984301,35.007500,1.38770,0.2838,100.0,96.498100,55.711498,56.787102,173.797607,836.563171,7.095300,36.943001,307.693512,1106.101318,14.06300,14.0472,407.514465,91.059196,93.308998,12.0,12.2238,1.3330,0.20460,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.6530,-4.5053,...,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2
6,0.0311,1.0392,406.091705,1104.010986,32.0,0.0000,36.943001,35.020100,1.37610,0.2838,100.0,96.490501,55.692299,56.787102,172.080994,835.258423,6.713800,34.956799,307.884186,1105.463501,14.07720,14.0396,407.412292,90.892403,93.301399,12.0,12.2162,1.3322,0.20690,36.184700,37.100201,28.024700,1.0,0.0,1.0,0.0,793.0,0.0,11.7219,-3.8183,...,0,406.09171,406.09171,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

time: 967 ms


# **trend 123 trend로 합침**

In [14]:
# A1_trend 계산
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 1
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 2
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 3
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 4
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 5
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 6
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 7
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 8
x_train.loc[((x_train.A1_trend1 == 1) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 9
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 10
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 11
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 12
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 13
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 14
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 15
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 16
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 17
x_train.loc[((x_train.A1_trend1 == 2) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 18
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 19
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 20
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 1) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 21
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 22
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 23
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 2) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 24
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 1)), 'A1_trend'] = 25
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 2)), 'A1_trend'] = 26
x_train.loc[((x_train.A1_trend1 == 3) & (x_train.A1_trend2 == 3) & (x_train.A1_trend3 == 3)), 'A1_trend'] = 27

# A2_trend 계산
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 1
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 2
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 3
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 4
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 5
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 6
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 7
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 8
x_train.loc[((x_train.A2_trend1 == 1) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 9
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 10
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 11
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 12
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 13
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 14
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 15
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 16
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 17
x_train.loc[((x_train.A2_trend1 == 2) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 18
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 19
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 20
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 1) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 21
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 22
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 23
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 2) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 24
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 1)), 'A2_trend'] = 25
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 2)), 'A2_trend'] = 26
x_train.loc[((x_train.A2_trend1 == 3) & (x_train.A2_trend2 == 3) & (x_train.A2_trend3 == 3)), 'A2_trend'] = 27

# A3_trend 계산
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 1)), 'N_trend'] = 1
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 2)), 'N_trend'] = 2
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 3)), 'N_trend'] = 3
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 1)), 'N_trend'] = 4
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 2)), 'N_trend'] = 5
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 3)), 'N_trend'] = 6
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 1)), 'N_trend'] = 7
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 2)), 'N_trend'] = 8
x_train.loc[((x_train.N_trend1 == 1) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 3)), 'N_trend'] = 9
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 1)), 'N_trend'] = 10
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 2)), 'N_trend'] = 11
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 3)), 'N_trend'] = 12
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 1)), 'N_trend'] = 13
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 2)), 'N_trend'] = 14
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 3)), 'N_trend'] = 15
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 1)), 'N_trend'] = 16
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 2)), 'N_trend'] = 17
x_train.loc[((x_train.N_trend1 == 2) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 3)), 'N_trend'] = 18
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 1)), 'N_trend'] = 19
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 2)), 'N_trend'] = 20
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 1) & (x_train.N_trend3 == 3)), 'N_trend'] = 21
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 1)), 'N_trend'] = 22
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 2)), 'N_trend'] = 23
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 2) & (x_train.N_trend3 == 3)), 'N_trend'] = 24
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 1)), 'N_trend'] = 25
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 2)), 'N_trend'] = 26
x_train.loc[((x_train.N_trend1 == 3) & (x_train.N_trend2 == 3) & (x_train.N_trend3 == 3)), 'N_trend'] = 27

# drop 된 값 있으므로 reset_index
x_train.reset_index(drop=True, inplace=True)
x_train


,P1.B2004,P1.B2016,P1.B3004,P1.B3005,P1.B4002,P1.B4005,P1.B400B,P1.B4022,P1.FCV01D,P1.FCV01Z,P1.FCV02D,P1.FCV02Z,P1.FCV03D,P1.FCV03Z,P1.FT01,P1.FT01Z,P1.FT02,P1.FT02Z,P1.FT03,P1.FT03Z,P1.LCV01D,P1.LCV01Z,P1.LIT01,P1.PCV01D,P1.PCV01Z,P1.PCV02D,P1.PCV02Z,P1.PIT01,P1.PIT02,P1.TIT01,P1.TIT02,P2.24Vdc,P2.Auto,P2.Emgy,P2.On,P2.SD01,P2.SIT01,P2.TripEx,P2.VT01e,P2.VXT02,...,A1_5mean,A1_5std,A1_3mean,A2_5min,A2_5max,A2_5mean,A2_5std,A2_3mean,N_5min,N_5max,N_5mean,N_5std,N_3mean,A1_3mean1,A1_3mean2,A1_3mean3,A2_3mean1,A2_3mean2,A2_3mean3,N_3mean1,N_3mean2,N_3mean3,A1_bound_up,A1_bound_down,A2_bound_up,A2_bound_down,N_bound_up,N_bound_down,A1_trend1,A1_trend2,A1_trend3,A2_trend1,A2_trend2,A2_trend3,N_trend1,N_trend2,N_trend3,A1_trend,A2_trend,N_trend
0,0.0311,1.0407,406.091705,1104.010986,32.0,0.0000,39.922100,35.023899,1.46590,0.2838,100.0,96.490501,55.745499,56.794701,171.508804,837.432983,6.523100,33.963799,307.312012,1105.144653,14.09210,14.0472,407.565613,91.525703,94.346603,12.0,12.2238,1.3322,0.20770,36.199902,37.100201,28.022100,1.0,0.0,1.0,0.0,793.0,0.0,11.8043,-3.7168,...,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2,14.0,14.0,14.0
1,0.0311,1.0434,406.091705,1104.010986,32.0,0.0000,33.963799,35.030701,1.45520,0.2838,100.0,96.414200,55.822601,56.794701,172.653198,830.039978,5.950900,30.984301,306.167603,1104.825562,14.11300,14.0472,407.463287,91.369003,93.415802,12.0,12.2238,1.3330,0.20920,36.184700,37.115501,28.027300,1.0,0.0,1.0,0.0,793.0,0.0,11.8339,-4.2946,...,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2,14.0,14.0,14.0
2,0.0311,1.0378,406.091705,1104.010986,32.0,0.0000,33.963799,35.016602,1.49180,0.2838,100.0,96.490501,55.693001,56.794701,174.369797,832.649292,7.095300,36.943001,308.074890,1102.912231,14.20050,14.0472,407.514465,91.233597,93.301399,12.0,12.2238,1.3330,0.20230,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.7272,-4.8579,...,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2,14.0,14.0,14.0
3,0.0311,1.0341,406.091705,1104.010986,32.0,0.0000,30.984301,35.007500,1.38770,0.2838,100.0,96.498100,55.711498,56.787102,173.797607,836.563171,7.095300,36.943001,307.693512,1106.101318,14.06300,14.0472,407.514465,91.059196,93.308998,12.0,12.2238,1.3330,0.20460,36.184700,37.100201,28.022900,1.0,0.0,1.0,0.0,793.0,0.0,11.6530,-4.5053,...,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2,14.0,14.0,14.0
4,0.0311,1.0392,406.091705,1104.010986,32.0,0.0000,36.943001,35.020100,1.37610,0.2838,100.0,96.490501,55.692299,56.787102,172.080994,835.258423,6.713800,34.956799,307.884186,1105.463501,14.07720,14.0396,407.412292,90.892403,93.301399,12.0,12.2162,1.3322,0.20690,36.184700,37.100201,28.024700,1.0,0.0,1.0,0.0,793.0,0.0,11.7219,-3.8183,...,406.09171,0.0,406.09171,1104.01099,1104.01099,1104.01099,0.0,1104.01099,32.0,32.0,32.0,0.0,32.0,406.09171,406.09171,406.09171,1104.01099,1104.01099,1104.01099,32.0,32.0,32.0,406.09171,406.09171,1104.01099,1104.01099,32.0,32.0,2,2,2,2,2,2,2,2,2,14.0,14.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

time: 1.81 s


# **y_train 업데이트**
- 과정에서 drop된 데이터가 있으므로


In [15]:
y_train = pd.DataFrame(data=x_train['attack'], columns=["attack"])
y_train = np.ravel(y_train)

y_train

array([0, 0, 0, ..., 0, 0, 0])

time: 90.8 ms


# **x_train에 PCA 처리**
- PCA 0.98

In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# features to learn
features = ['P1.B2004', 'P1.B2016', 'P1.B3004', 'P1.B3005',\
'P1.B4002', 'P1.B4005', 'P1.B400B', 'P1.B4022', 'P1.FCV01D',\
'P1.FCV01Z', 'P1.FCV02D', 'P1.FCV02Z', 'P1.FCV03D', 'P1.FCV03Z',\
'P1.FT01', 'P1.FT01Z', 'P1.FT02', 'P1.FT02Z', 'P1.FT03',\
'P1.FT03Z', 'P1.LCV01D', 'P1.LCV01Z', 'P1.LIT01', 'P1.PCV01D',\
'P1.PCV01Z', 'P1.PCV02D', 'P1.PCV02Z', 'P1.PIT01', 'P1.PIT02',\
'P1.TIT01', 'P1.TIT02', 'P2.24Vdc', 'P2.Auto', 'P2.Emgy',\
'P2.On', 'P2.SD01', 'P2.SIT01', 'P2.TripEx', 'P2.VT01e',\
'P2.VXT02', 'P2.VXT03', 'P2.VYT02', 'P2.VYT03', 'P3.LCP01D', \
'P3.LCV01D', 'P3.LH', 'P3.LL', 'P3.LT01', 'P4.HT_FD',\
'P4.HT_LD', 'P4.HT_PO', 'P4.HT_PS', 'P4.LD', 'P4.ST_FD',\
'P4.ST_LD', 'P4.ST_PO', 'P4.ST_PS', 'P4.ST_PT01', 'P4.ST_TT01',
'A1_trend', 'A2_trend', 'N_trend',
]

#scaling
scaler = StandardScaler()
scaler.fit(x_train[features])
scaled_x_train = scaler.transform(x_train[features])

# PCA 적용
pca_model = PCA(n_components=33)
pca_model.fit(scaled_x_train)
train_pca = pca_model.transform(scaled_x_train)

print("transformed x_train: {}".format(str(train_pca.shape)))

transformed x_train: (727194, 33)
time: 11.1 s


# **x_test에도 동일한 작업**

In [0]:
x_test.drop(['time'], axis=1, inplace=True)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
test_pca = pca_model.transform(scaled_x_test)






time: 1.68 s


# **XGB 학습**

In [0]:
# Xgboost 학습
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=2, learning_rate=0.1, 
                        max_depth=8, min_child_weight=10, gamma= 0, random_state=0)
clf.fit(train_pca, y_train, verbose =1)

y_pred = clf.predict(test_pca)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

#save model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
pickle.dump(clf, open(path + filename, "wb"))

86.33303262998085 

confusion matrix
[[151196   1688]
 [ 23158   5754]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92    152884
           1       0.77      0.20      0.32     28912

    accuracy                           0.86    181796
   macro avg       0.82      0.59      0.62    181796
weighted avg       0.85      0.86      0.83    181796

time: 16min 52s


# **N**

# **5%**

In [17]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
85.91333142643403 

confusion matrix
[[150434   2450]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92    152884
           1       0.70      0.20      0.31     28912

    accuracy                           0.86    181796
   macro avg       0.78      0.59      0.62    181796
weighted avg       0.84      0.86      0.82    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
852
normal error인데 normal으로 분류된 경우
6792
오분류 에러: 11.145996860282574
time: 10.3 s


10

In [18]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
85.50298136372638 

confusion matrix
[[149688   3196]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92    152884
           1       0.64      0.20      0.30     28912

    accuracy                           0.86    181796
   macro avg       0.75      0.59      0.61    181796
weighted avg       0.83      0.86      0.82    181796

error 갯수
15287
normal error인데 attack으로 분류된 경우
1686
normal error인데 normal으로 분류된 경우
13601
오분류 에러: 11.028978870936088
time: 10.8 s


15

In [20]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
85.0981319721006 

confusion matrix
[[148952   3932]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.97      0.92    152884
           1       0.59      0.20      0.30     28912

    accuracy                           0.85    181796
   macro avg       0.73      0.59      0.61    181796
weighted avg       0.82      0.85      0.82    181796

error 갯수
22931
normal error인데 attack으로 분류된 경우
2464
normal error인데 normal으로 분류된 경우
20467
오분류 에러: 10.745279316209498
time: 9.47 s


30

In [21]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
83.8687319853022 

confusion matrix
[[146717   6167]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.91    152884
           1       0.48      0.20      0.28     28912

    accuracy                           0.84    181796
   macro avg       0.67      0.58      0.60    181796
weighted avg       0.80      0.84      0.81    181796

error 갯수
45866
normal error인데 attack으로 분류된 경우
4857
normal error인데 normal으로 분류된 경우
41009
오분류 에러: 10.589543452666463
time: 16.5 s


# **A1**

# **5%**

In [0]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
85.32365948645734 

confusion matrix
[[149362   3522]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92    152884
           1       0.62      0.20      0.30     28912

    accuracy                           0.85    181796
   macro avg       0.74      0.59      0.61    181796
weighted avg       0.83      0.85      0.82    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
1928
normal error인데 normal으로 분류된 경우
5716
오분류 에러: 25.222396650968083
time: 9.43 s


10

In [22]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
84.32583775220577 

confusion matrix
[[147548   5336]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.97      0.91    152884
           1       0.52      0.20      0.29     28912

    accuracy                           0.84    181796
   macro avg       0.69      0.58      0.60    181796
weighted avg       0.81      0.84      0.81    181796

error 갯수
15288
normal error인데 attack으로 분류된 경우
3838
normal error인데 normal으로 분류된 경우
11450
오분류 에러: 25.10465724751439
time: 13.4 s


15

In [23]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
83.26530836762085 

confusion matrix
[[145621   7263]
 [ 23160   5752]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91    152884
           1       0.44      0.20      0.27     28912

    accuracy                           0.83    181796
   macro avg       0.65      0.58      0.59    181796
weighted avg       0.80      0.83      0.81    181796

error 갯수
22931
normal error인데 attack으로 분류된 경우
5833
normal error인데 normal으로 분류된 경우
17098
오분류 에러: 25.437181108543022
time: 11.2 s


30

In [24]:
path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
80.16347994455325 

confusion matrix
[[139982  12902]
 [ 23160   5752]]
              precision    recall  f1-score   support

           0       0.86      0.92      0.89    152884
           1       0.31      0.20      0.24     28912

    accuracy                           0.80    181796
   macro avg       0.58      0.56      0.56    181796
weighted avg       0.77      0.80      0.78    181796

error 갯수
45865
normal error인데 attack으로 분류된 경우
11681
normal error인데 normal으로 분류된 경우
34184
오분류 에러: 25.468221955739672
time: 15.8 s


# **A2**

5

In [0]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
86.22906994653347 

confusion matrix
[[151008   1876]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92    152884
           1       0.75      0.20      0.31     28912

    accuracy                           0.86    181796
   macro avg       0.81      0.59      0.62    181796
weighted avg       0.85      0.86      0.83    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
281
normal error인데 normal으로 분류된 경우
7363
오분류 에러: 3.676085818942962
time: 9.04 s


10

In [25]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
86.13720873946622 

confusion matrix
[[150841   2043]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92    152884
           1       0.74      0.20      0.31     28912

    accuracy                           0.86    181796
   macro avg       0.80      0.59      0.62    181796
weighted avg       0.85      0.86      0.83    181796

error 갯수
15287
normal error인데 attack으로 분류된 경우
521
normal error인데 normal으로 분류된 경우
14766
오분류 에러: 3.4081245502714728
time: 10 s


15

In [26]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
86.02389491517965 

confusion matrix
[[150635   2249]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92    152884
           1       0.72      0.20      0.31     28912

    accuracy                           0.86    181796
   macro avg       0.79      0.59      0.62    181796
weighted avg       0.84      0.86      0.83    181796

error 갯수
22933
normal error인데 attack으로 분류된 경우
790
normal error인데 normal으로 분류된 경우
22143
오분류 에러: 3.44481751188244
time: 12.3 s


30

In [27]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
85.70210565689014 

confusion matrix
[[150050   2834]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92    152884
           1       0.67      0.20      0.31     28912

    accuracy                           0.86    181796
   macro avg       0.77      0.59      0.61    181796
weighted avg       0.84      0.86      0.82    181796

error 갯수
45865
normal error인데 attack으로 분류된 경우
1631
normal error인데 normal으로 분류된 경우
44234
오분류 에러: 3.556088520658454
time: 16.9 s


# **A1 N**

5

In [0]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
84.61352284978767 

confusion matrix
[[148071   4813]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.97      0.91    152884
           1       0.54      0.20      0.29     28912

    accuracy                           0.85    181796
   macro avg       0.70      0.58      0.60    181796
weighted avg       0.81      0.85      0.81    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
3197
normal error인데 normal으로 분류된 경우
4447
오분류 에러: 41.82365253793825
time: 11.6 s


10

In [28]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
82.93416796849216 

confusion matrix
[[145018   7866]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90    152884
           1       0.42      0.20      0.27     28912

    accuracy                           0.83    181796
   macro avg       0.64      0.57      0.59    181796
weighted avg       0.79      0.83      0.80    181796

error 갯수
15287
normal error인데 attack으로 분류된 경우
6343
normal error인데 normal으로 분류된 경우
8944
오분류 에러: 41.49277163603061
time: 9.65 s


15

In [29]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
81.22345926202996 

confusion matrix
[[141908  10976]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.93      0.89    152884
           1       0.34      0.20      0.25     28912

    accuracy                           0.81    181796
   macro avg       0.60      0.56      0.57    181796
weighted avg       0.78      0.81      0.79    181796

error 갯수
22932
normal error인데 attack으로 분류된 경우
9495
normal error인데 normal으로 분류된 경우
13437
오분류 에러: 41.405023547880695
time: 11 s


30

In [30]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
76.02807542520188 

confusion matrix
[[132463  20421]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86    152884
           1       0.22      0.20      0.21     28912

    accuracy                           0.76    181796
   macro avg       0.54      0.53      0.53    181796
weighted avg       0.75      0.76      0.76    181796

error 갯수
45863
normal error인데 attack으로 분류된 경우
19063
normal error인데 normal으로 분류된 경우
26800
오분류 에러: 41.565096046922356
time: 15 s


# **A2 N**

5

In [0]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
85.91333142643403 

confusion matrix
[[150434   2450]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92    152884
           1       0.70      0.20      0.31     28912

    accuracy                           0.86    181796
   macro avg       0.78      0.59      0.62    181796
weighted avg       0.84      0.86      0.82    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
847
normal error인데 normal으로 분류된 경우
6797
오분류 에러: 11.08058608058608
time: 8.34 s


10

In [32]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
85.4633765319369 

confusion matrix
[[149616   3268]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.98      0.92    152884
           1       0.64      0.20      0.30     28912

    accuracy                           0.85    181796
   macro avg       0.75      0.59      0.61    181796
weighted avg       0.83      0.85      0.82    181796

error 갯수
15287
normal error인데 attack으로 분류된 경우
1727
normal error인데 normal으로 분류된 경우
13560
오분류 에러: 11.297180610976646
time: 7.91 s


15

In [33]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
85.09153116680235 

confusion matrix
[[148940   3944]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.87      0.97      0.92    152884
           1       0.59      0.20      0.30     28912

    accuracy                           0.85    181796
   macro avg       0.73      0.59      0.61    181796
weighted avg       0.82      0.85      0.82    181796

error 갯수
22933
normal error인데 attack으로 분류된 경우
2496
normal error인데 normal으로 분류된 경우
20437
오분류 에러: 10.88387912615009
time: 11.3 s


30

In [34]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
83.77577064401856 

confusion matrix
[[146548   6336]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.91    152884
           1       0.48      0.20      0.28     28912

    accuracy                           0.84    181796
   macro avg       0.67      0.58      0.59    181796
weighted avg       0.80      0.84      0.81    181796

error 갯수
45864
normal error인데 attack으로 분류된 경우
4895
normal error인데 normal으로 분류된 경우
40969
오분류 에러: 10.672858887144601
time: 16.7 s


# **A1 A2**

5

In [0]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
84.60032123919117 

confusion matrix
[[148047   4837]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.97      0.91    152884
           1       0.54      0.20      0.29     28912

    accuracy                           0.85    181796
   macro avg       0.70      0.58      0.60    181796
weighted avg       0.81      0.85      0.81    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
3234
normal error인데 normal으로 분류된 경우
4410
오분류 에러: 42.30769230769231
time: 12 s


10

In [35]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
82.96882219630794 

confusion matrix
[[145081   7803]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90    152884
           1       0.42      0.20      0.27     28912

    accuracy                           0.83    181796
   macro avg       0.64      0.57      0.59    181796
weighted avg       0.79      0.83      0.80    181796

error 갯수
15287
normal error인데 attack으로 분류된 경우
6286
normal error인데 normal으로 분류된 경우
9001
오분류 에러: 41.11990580231569
time: 9.71 s


15

In [37]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
81.33622301920835 

confusion matrix
[[142113  10771]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.93      0.89    152884
           1       0.35      0.20      0.25     28912

    accuracy                           0.81    181796
   macro avg       0.60      0.56      0.57    181796
weighted avg       0.78      0.81      0.79    181796

error 갯수
22933
normal error인데 attack으로 분류된 경우
9343
normal error인데 normal으로 분류된 경우
13590
오분류 에러: 40.74041773862992
time: 11 s


30

In [38]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
76.38561904552355 

confusion matrix
[[133113  19771]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86    152884
           1       0.23      0.20      0.21     28912

    accuracy                           0.76    181796
   macro avg       0.54      0.53      0.54    181796
weighted avg       0.75      0.76      0.76    181796

error 갯수
45864
normal error인데 attack으로 분류된 경우
18544
normal error인데 normal으로 분류된 경우
27320
오분류 에러: 40.432583289726146
time: 15 s


# **A1 A2 N**

5

In [0]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_5percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
174156
에러 O
7644
83.42537789610333 

confusion matrix
[[145911   6973]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91    152884
           1       0.45      0.20      0.28     28912

    accuracy                           0.83    181796
   macro avg       0.66      0.58      0.59    181796
weighted avg       0.80      0.83      0.81    181796

error 갯수
7644
normal error인데 attack으로 분류된 경우
5373
normal error인데 normal으로 분류된 경우
2271
오분류 에러: 70.29042386185243
time: 8.3 s


10

In [39]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_10percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
166512
에러 O
15288
80.6590904090299 

confusion matrix
[[140882  12002]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.86      0.92      0.89    152884
           1       0.32      0.20      0.25     28912

    accuracy                           0.81    181796
   macro avg       0.59      0.56      0.57    181796
weighted avg       0.77      0.81      0.79    181796

error 갯수
15288
normal error인데 attack으로 분류된 경우
10483
normal error인데 normal으로 분류된 경우
4805
오분류 에러: 68.57012035583465
time: 9.22 s


15

In [40]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_15percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
158867
에러 O
22933
77.92800721688046 

confusion matrix
[[135917  16967]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.85      0.89      0.87    152884
           1       0.25      0.20      0.22     28912

    accuracy                           0.78    181796
   macro avg       0.55      0.54      0.55    181796
weighted avg       0.76      0.78      0.77    181796

error 갯수
22931
normal error인데 attack으로 분류된 경우
15489
normal error인데 normal으로 분류된 경우
7442
오분류 에러: 67.54611661070167
time: 13.2 s


30

In [41]:


path = os.getcwd() + "/drive/My Drive/data/AAN_new/A1_A2_N"
x_test = pd.read_pickle(path + '/reset_Error_whole_x_test_30percent.pickle')

print("데이터 분포")
num0=0
num1=0
for i in x_test["error"]:
  if(i==0):
    num0 = num0+1
  elif(i==1):
    num1 = num1+1
print('에러 X')
print(num0)
print('에러 O')
print(num1)

# x_test의 A1, A2, N의 rolling min, max, mean, std 정보추가
x_test['A1_5min'] = x_test['P1.B3004'].rolling(window=5).min()
x_test['A1_5max'] = x_test['P1.B3004'].rolling(window=5).max()
x_test['A1_5mean'] = x_test['P1.B3004'].rolling(window=5).mean()
x_test['A1_5std'] = x_test['P1.B3004'].rolling(window=5).std()
x_test['A1_3mean'] = x_test['P1.B3004'].rolling(window=3).mean()
x_test['A2_5min'] = x_test['P1.B3005'].rolling(window=5).min()
x_test['A2_5max'] = x_test['P1.B3005'].rolling(window=5).max()
x_test['A2_5mean'] = x_test['P1.B3005'].rolling(window=5).mean()
x_test['A2_5std'] = x_test['P1.B3005'].rolling(window=5).std()
x_test['A2_3mean'] = x_test['P1.B3005'].rolling(window=3).mean()
x_test['N_5min'] = x_test['P1.B4002'].rolling(window=5).min()
x_test['N_5max'] = x_test['P1.B4002'].rolling(window=5).max()
x_test['N_5mean'] = x_test['P1.B4002'].rolling(window=5).mean()
x_test['N_5std'] = x_test['P1.B4002'].rolling(window=5).std()
x_test['N_3mean'] = x_test['P1.B4002'].rolling(window=3).mean()
# rolling NaN 제거
x_test.drop(x_test.head(2).index, inplace=True)
x_test.reset_index(drop=True, inplace=True)

# 한줄로 만듬.
lastindex = 181799
x_test['A1_3mean1'] = 999.0
x_test['A1_3mean2'] = 999.0
x_test['A1_3mean3'] = x_test['A1_3mean']  # 보기편하게
x_test['A2_3mean1'] = 999.0
x_test['A2_3mean2'] = 999.0
x_test['A2_3mean3'] = x_test['A2_3mean']  # 보기편하게
x_test['N_3mean1'] = 999.0
x_test['N_3mean2'] = 999.0
x_test['N_3mean3'] = x_test['N_3mean']  # 보기편하게

# 루프없이 이전, 그 이전 3mean 값을 모으기위해서 테이블끼리 붙임.
df = x_test
df2 = x_test
ddf = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf2 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf = ddf.append(ddf2, ignore_index=True)
df = ddf.append(df, ignore_index=True)
x_test['A1_3mean1'] = df['A1_3mean3']
x_test['A2_3mean1'] = df['A2_3mean3']
x_test['N_3mean1'] = df['N_3mean3']
df2 = df2.iloc[1:]  # remove top
ddf3 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf4 = pd.DataFrame([[np.nan] * len(df.columns)], columns = df.columns) # 여백용
ddf3 = ddf3.append(ddf4, ignore_index=True)
df2 = ddf3.append(df2, ignore_index=True)
x_test['A1_3mean2'] = df2['A1_3mean3']
x_test['A2_3mean2'] = df2['A2_3mean3']
x_test['N_3mean2'] = df2['N_3mean3']

# bound_up, bound_down 계산
x_test['A1_bound_up'] = x_test['A1_5mean'] + x_test['A1_5std']/2 
x_test['A1_bound_down'] = x_test['A1_5mean'] - x_test['A1_5std']/2 
x_test['A2_bound_up'] = x_test['A2_5mean'] + x_test['A2_5std']/2 
x_test['A2_bound_down'] = x_test['A2_5mean'] - x_test['A2_5std']/2 
x_test['N_bound_up'] = x_test['N_5mean'] + x_test['N_5std']/2 
x_test['N_bound_down'] = x_test['N_5mean'] - x_test['N_5std']/2
# 소수점 6번째에서 반올림. 원래 데이터의 유효숫자 5.
x_test = x_test.apply(lambda x: pd.Series.round(x, 5))
x_test['A1_trend1'] = 999
x_test['A1_trend2'] = 999
x_test['A1_trend3'] = 999
x_test['A2_trend1'] = 999
x_test['A2_trend2'] = 999
x_test['A2_trend3'] = 999
x_test['N_trend1'] = 999
x_test['N_trend2'] = 999
x_test['N_trend3'] = 999
# NaN으로 못쓰는 데이터 제거
x_test.drop(x_test.head(2).index, inplace=True)
# A1, A2, N의 trend1,2,3 구함
cal_trendA1(x_test, x_test['A1_bound_down'], x_test['A1_bound_up'], x_test['A1_3mean1'], x_test['A1_3mean2'], x_test['A1_3mean3'])
cal_trendA2(x_test, x_test['A2_bound_down'], x_test['A2_bound_up'], x_test['A2_3mean1'], x_test['A2_3mean2'], x_test['A2_3mean3'])
cal_trendN(x_test, x_test['N_bound_down'], x_test['N_bound_up'], x_test['N_3mean1'], x_test['N_3mean2'], x_test['N_3mean3'])

# A1_trend 계산
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 1
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 2
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 3
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 4
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 5
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 6
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 7
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 8
x_test.loc[((x_test.A1_trend1 == 1) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 9
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 10
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 11
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 12
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 13
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 14
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 15
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 16
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 17
x_test.loc[((x_test.A1_trend1 == 2) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 18
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 19
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 20
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 1) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 21
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 22
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 23
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 2) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 24
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 1)), 'A1_trend'] = 25
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 2)), 'A1_trend'] = 26
x_test.loc[((x_test.A1_trend1 == 3) & (x_test.A1_trend2 == 3) & (x_test.A1_trend3 == 3)), 'A1_trend'] = 27
# A2_trend 계산
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 1
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 2
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 3
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 4
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 5
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 6
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 7
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 8
x_test.loc[((x_test.A2_trend1 == 1) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 9
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 10
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 11
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 12
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 13
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 14
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 15
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 16
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 17
x_test.loc[((x_test.A2_trend1 == 2) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 18
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 19
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 20
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 1) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 21
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 22
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 23
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 2) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 24
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 1)), 'A2_trend'] = 25
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 2)), 'A2_trend'] = 26
x_test.loc[((x_test.A2_trend1 == 3) & (x_test.A2_trend2 == 3) & (x_test.A2_trend3 == 3)), 'A2_trend'] = 27
# A3_trend 계산
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 1
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 2
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 3
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 4
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 5
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 6
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 7
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 8
x_test.loc[((x_test.N_trend1 == 1) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 9
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 10
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 11
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 12
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 13
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 14
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 15
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 16
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 17
x_test.loc[((x_test.N_trend1 == 2) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 18
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 1)), 'N_trend'] = 19
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 2)), 'N_trend'] = 20
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 1) & (x_test.N_trend3 == 3)), 'N_trend'] = 21
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 1)), 'N_trend'] = 22
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 2)), 'N_trend'] = 23
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 2) & (x_test.N_trend3 == 3)), 'N_trend'] = 24
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 1)), 'N_trend'] = 25
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 2)), 'N_trend'] = 26
x_test.loc[((x_test.N_trend1 == 3) & (x_test.N_trend2 == 3) & (x_test.N_trend3 == 3)), 'N_trend'] = 27
# drop 된 값 있으므로 reset_index
x_test.reset_index(drop=True, inplace=True)
# y_test 다시 만듬
y_test = pd.DataFrame(data=x_test['attack'], columns=["attack"])
y_test = np.ravel(y_test)
#scaling
scaled_x_test = scaler.transform(x_test[features])
# PCA 적용
pca_x_test = pca_model.transform(scaled_x_test)

#load model
path = F"/content/drive/My Drive/data/model/"
filename = 'AAN_new_final_trend+PCA+XGB.pkl'
loaded_model = pickle.load(open(path+ filename, "rb"))
y_pred = loaded_model.predict(pca_x_test)

# accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc*100, '\n')
# print confusion matrix
print('confusion matrix')
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

# prediction 값.
df = pd.DataFrame(data=y_pred, columns=["attack"])
a = 0
b = 0
errors=0
for i, row in enumerate(x_test["error"]):
  if(row==1):
    errors = errors+1
    if(df.iloc[i].item()==1): # normal error인데 attack으로 분류된 경우.
      a=a+1
    elif(df.iloc[i].item()==0):  #normal error인데 normal으로 분류된 경우.
      b=b+1

print('error 갯수')
print(errors)
print('normal error인데 attack으로 분류된 경우')
print(a)
print('normal error인데 normal으로 분류된 경우')
print(b)
print('오분류 에러: {}'.format(str(a/errors*100)))




데이터 분포
에러 X
135934
에러 O
45866
69.51033026029175 

confusion matrix
[[120614  32270]
 [ 23159   5753]]
              precision    recall  f1-score   support

           0       0.84      0.79      0.81    152884
           1       0.15      0.20      0.17     28912

    accuracy                           0.70    181796
   macro avg       0.50      0.49      0.49    181796
weighted avg       0.73      0.70      0.71    181796

error 갯수
45863
normal error인데 attack으로 분류된 경우
30755
normal error인데 normal으로 분류된 경우
15108
오분류 에러: 67.05841309988443
time: 14.3 s
